<a href="https://colab.research.google.com/github/Darally06/Labo_arbol/blob/main/Copia_de_LAB1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LABORATORIO 1:
### *ESTRUCTURA DE DATOS 2*
Por: Juan Aguirre, Daniela Guerra, Maria Gabriela Rey.

Marzo 17, 2024


### 1. Librerias

In [1]:
from google.colab import files
from collections import deque
import os
import shutil
import re
from PIL import Image
from typing import Any, List, Optional, Tuple
from graphviz import Digraph
from IPython.display import Image
from ast import NodeTransformer

### 2. Manejo de las imágenes


In [2]:
from google.colab import files
uploaded = files.upload()

In [3]:
# Guardar nombres de imagenes
def obtener_categoria(name):
    try:
        categorias = {
            "dog": "dogs",
            "rider": "humans",
            "horse": "horses",
            "cat": "cats",
            "bike": "bikes",
            "car": "cars"
        }
        for key, value in categorias.items():
            if key in name.lower():
                return value
        return "flowers"
    except Exception as e:
        print(f"Error al obtener la categoría: {e}")
        return None

def organizar_carpetas(destino, carpetas):
    try:
        # Crear carpeta general
        if not os.path.exists(destino):
            os.makedirs(destino)

        # Categorías
        for categoria in carpetas:
            carpeta = os.path.join(destino, categoria)
            if not os.path.exists(carpeta):
                os.makedirs(carpeta)

        archivos = os.listdir(destino)
        for archivo in archivos:
            ruta = os.path.join(destino, archivo)
            if os.path.isfile(ruta):
                categoria = obtener_categoria(archivo)
                c_destino = os.path.join(destino, categoria)
                shutil.move(ruta, os.path.join(c_destino, archivo))
    except Exception as e:
        print(f"Error al organizar las carpetas: {e}")

# Procesar imagenes
def procesar_imagenes(destino, carpetas):
    try:
        imagenes = {}
        for categoria in carpetas:
            carpeta = os.path.join(destino, categoria)
            imagenes[categoria] = []
            archivos = os.listdir(carpeta)
            for archivo in archivos:
                ruta_archivo = os.path.join(carpeta, archivo)
                if os.path.isfile(ruta_archivo):
                    name = os.path.basename(ruta_archivo)
                    peso_archivo = os.path.getsize(ruta_archivo) / (1024 * 1024)
                    ruta = os.path.join(ruta_archivo, name)
                    registro = {"Nombre": name, "Categoria": categoria, "Peso": peso_archivo, "Ruta": ruta}
                    imagenes[categoria].append(registro)
        return imagenes
    except Exception as e:
        print(f"Error al procesar las imágenes: {e}")
        return {}

### 4. Crear Arbol AVL

In [4]:
class Node:
  def __init__(self, nombre):
    self.nombre = nombre
    self.izq = None
    self.der = None
    self.padre = None
    self.altura = 1

class AVL:
    def __init__(self):
        self.raiz = None

    # REEQUILIBRIO
    def altura (self, nodo):
        if nodo is None:
            return 0
        return nodo.altura

    def factor (self, nodo):
        if nodo is None:
            return 0
        return  self.altura(nodo.der) - self.altura(nodo.izq)

    # Rotaciones
    def rotar_derecha(self, nodo):
        nodo_izquierdo = nodo.izq

        T2 = nodo_izquierdo.der

        nodo_izquierdo.der = nodo
        nodo.izq= T2

        nodo.altura = 1 + max(self.altura(nodo.izq), self.altura(nodo.der))
        nodo_izquierdo.altura = 1 + max(self.altura(nodo_izquierdo.izq), self.altura(nodo_izquierdo.der))
        return nodo_izquierdo

    def rotar_izquierda(self, nodo):
        nodo_derecho = nodo.der
        if nodo_derecho is None:
            return nodo

        T3 = nodo_derecho.izq

        nodo_derecho.izq = nodo
        nodo.der = T3

        nodo.altura = 1 + max(self.altura(nodo.izq), self.altura(nodo.der))
        nodo_derecho.altura = 1 + max(self.altura(nodo_derecho.izq), self.altura(nodo_derecho.der))

        return nodo_derecho

    def insertar(self, nombre):
        self.raiz = self._insertar_recur(self.raiz, nombre)

    def _insertar_recur(self, raiz, nombre, padre=None):
        if raiz is None:
            return Node(nombre)

        if nombre < raiz.nombre:
            if raiz.izq is None:
                raiz.izq = Node(nombre)
                raiz.izq.padre = raiz
            else:
                raiz.izq = self._insertar_recur(raiz.izq, nombre, raiz)
        else:
            if raiz.der is None:
                raiz.der = Node(nombre)
                raiz.der.padre = raiz
            else:
                raiz.der = self._insertar_recur(raiz.der, nombre, raiz)

        raiz.altura = 1 + max(self.altura(raiz.izq), self.altura(raiz.der))

        balance = self.factor(raiz)

        if balance > 1 and self.factor(raiz.izq) >= 0:
            return self.rotar_derecha(raiz)
        if balance < -1 and self.factor(raiz.der) <= 0:
            return self.rotar_izquierda(raiz)
        if balance > 1 and self.factor(raiz.izq) < 0:
            raiz.izq = self.rotar_izquierda(raiz.izq)
            return self.rotar_derecha(raiz)
        if balance < -1 and self.factor(raiz.der) > 0:
            raiz.der = self.rotar_derecha(raiz.der)
            return self.rotar_izquierda(raiz)

        return raiz

    def inorden(self, raiz):
        if raiz:
            self.inorden(raiz.izq)
            print({raiz.nombre})
            self.inorden(raiz.der)

    def vaciar_arbol(self):
        self.raiz = None

    def generar_dot_arbol(self, filename):
        dot = Digraph()

        def agregar_nodos_y_aristas(nodo):
            if nodo is not None:
                dot.node(str(nodo.nombre), label=str(nodo.nombre))

                if nodo.izq is not None:
                    dot.edge(str(nodo.nombre), str(nodo.izq.nombre))
                    agregar_nodos_y_aristas(nodo.izq)
                if nodo.der is not None:
                    dot.edge(str(nodo.nombre), str(nodo.der.nombre))
                    agregar_nodos_y_aristas(nodo.der)

        agregar_nodos_y_aristas(self.raiz)
        dot.render(filename, format='png')

    def buscar(self, nombre):
        return self._buscar_recur(self.raiz, nombre)

    def _buscar_recur(self, nodo, nombre):
        if nodo is None:
            return None
        elif nombre == nodo.nombre:
            return nodo
        elif nombre < nodo.nombre:
            return self._buscar_recur(nodo.izq, nombre)
        else:
            return self._buscar_recur(nodo.der, nombre)

    def nivel(self, nodo):
        if nodo:
            return self._obtener_nivel_recur(self.raiz, nodo, 1)
        else:
            return None

    def _obtener_nivel_recur(self, raiz, nodo, nivel_actual):
        if raiz is None:
            return None
        if raiz == nodo:
            return nivel_actual
        nivel_izq = self._obtener_nivel_recur(raiz.izq, nodo, nivel_actual + 1)
        if nivel_izq is not None:
            return nivel_izq
        nivel_der = self._obtener_nivel_recur(raiz.der, nodo, nivel_actual + 1)
        return nivel_der

    def obtener_padre(self, nodo):
        if nodo:
            return self._obtener_padre_recur(self.raiz, nodo, None)
        else:
            return None

    def _obtener_padre_recur(self, raiz, nodo, padre_actual):
        if raiz is None:
            return None
        elif raiz.izq == nodo or raiz.der == nodo:
            return raiz
        elif nodo.nombre < raiz.nombre:
            return self._obtener_padre_recur(raiz.izq, nodo, raiz)
        else:
            return self._obtener_padre_recur(raiz.der, nodo, raiz)

    def obtener_abuelo(self, nodo):
        if nodo and nodo.padre:
            return nodo.padre.padre.nombre if nodo.padre.padre else None
        return None

    def obtener_tio(self, nodo):
        if nodo and nodo.padre and nodo.padre.padre:
            if nodo.padre == nodo.padre.padre.izq:
                return nodo.padre.padre.der.nombre if nodo.padre.padre.der else None
            else:
                return nodo.padre.padre.izq.nombre if nodo.padre.padre.izq else None
        return None

    def recorrido(self):
        if self.raiz is None:
            print("El árbol AVL está vacío.")
            return

        queue = [self.raiz]

        while queue:
            nodo = queue.pop(0)
            print(nodo.nombre)

            if nodo.izq:
                queue.append(nodo.izq)
            if nodo.der:
                queue.append(nodo.der)

    # Eliminacion
    def eliminar(self, raiz, nombre_archivo, padre=None):
        if not raiz:
            return raiz

        if nombre_archivo < raiz.nombre:
            raiz.izq = self.eliminar(raiz.izq, nombre_archivo, raiz)
        elif nombre_archivo > raiz.nombre:
            raiz.der = self.eliminar(raiz.der, nombre_archivo, raiz)
        else:
            # Nodo con un solo hijo o sin hijos
            if raiz.izq is None:
                temp = raiz.der
                if padre and padre.izq== raiz:
                    padre.izq = temp
                else:
                    padre.der = temp
                raiz = None
                return temp
            elif raiz.der is None:
                temp = raiz.izq
                if padre and padre.izq == raiz:
                    padre.izq = temp
                else:
                    padre.der = temp
                raiz = None
                return temp

            temp = self.obtener_minimo(raiz.der)

            raiz.nombre = temp.nombre

            raiz.der = self.eliminar(raiz.der, temp.nombre, raiz)

        if raiz is None:
            return raiz

        raiz.altura = 1 + max(self.altura(raiz.izq), self.altura(raiz.der))

        balance = self.obtener_balance(raiz)

        if balance > 1 and self.obtener_balance(raiz.izq) >= 0:
            return self.rotar_derecha(raiz)
        if balance < -1 and self.obtener_balance(raiz.der) <= 0:
            return self.rotar_izquierda(raiz)
        if balance > 1 and self.obtener_balance(raiz.izq) < 0:
            raiz.izq = self.rotar_izquierda(raiz.izq)
            return self.rotar_derecha(raiz)
        if balance < -1 and self.obtener_balance(raiz.der) > 0:
            raiz.der= self.rotar_derecha(raiz.der)
            return self.rotar_izquierda(raiz)
        return raiz



### Algoritmo Principal (alpha)

In [5]:
# CREAR ARBOL
arbol = AVL()
raiz = None

# IMAGENES POR CARPETA
destino = "/content"
carpetas_exist = ["bike", "cars", "cats", "dogs", "flowers", "horses", "human"]
organizar_carpetas(destino, carpetas_exist)
imagenes = procesar_imagenes(destino, carpetas_exist)

def separador():
    print("-----------------------------")

def nombre_en_carpetas(nombre, destino):
    categorias = ["dogs", "flowers", "bikes", "humans", "horses", "cats", "cars"]
    for categoria in categorias:
        carpeta = os.path.join(destino, categoria)
        if os.path.exists(os.path.join(carpeta, nombre)):
            return True
    return False



#### op1

In [6]:
def option_1(arbol, destino):
    print("INSERTAR IMAGEN")
    insertados = []
    separador()

    while True:
        nuevo = input("Ingrese el nombre del nodo a insertar: ")

        if not nuevo:
            raise ValueError("El nombre del archivo no puede ser vacío")
        if not isinstance(nuevo, str):
           raise TypeError("El nombre del archivo debe ser una cadena de caracteres")
        separador()

        if nuevo in insertados:
            print("¡Esta imagen ya ha sido insertada! Por favor, ingrese otro nombre.")
            continue

        if nombre_en_carpetas(nuevo, destino):
            nodo = Node(nuevo)
            arbol.insertar(nuevo)
            arbol.inorden(arbol.raiz)
            arbol.generar_dot_arbol('arbol.dot')
            insertados.append(nuevo)
            print(f"Imagen '{nuevo}' insertada en el árbol.")

        else:
            print("El archivo no existe en ninguna de las carpetas.")

        while True:
            opcion_continuar = input("¿Desea insertar otra imagen? (si/no): ").lower()
            if opcion_continuar in ["si", "no"]:
                break
            else:
                print("Por favor, ingrese  sí o  no.")

        if opcion_continuar == "no":
            separador()
            break


#### op2


In [7]:
def option_2():
    print("ELIMINAR")
    separador()

    while True:
        if arbol.raiz is None:
            print("El árbol está vacío. No se puede eliminar ningún nodo.")
            separador()
            break

        nombre_archivo = input("Ingrese el nombre del nodo a eliminar: ")
        if not nombre_archivo:
            raise ValueError("El nombre del archivo no puede ser vacío")
        if not isinstance(nombre_archivo, str):
           raise TypeError("El nombre del archivo debe ser una cadena de caracteres")
        separador()

        nodo_encontrado = arbol.buscar(nombre_archivo)
        if nodo_encontrado:
            nodo = Node(nodo_encontrado)
            arbol.eliminar(nodo_encontrado)
            arbol.inorden(arbol.raiz)
            arbol.generar_dot_arbol('arbol.dot')
        else:
            print(f"No se encontró el nodo con nombre '{nombre_archivo}' en el árbol AVL.")

        while True:
            opcion = input("¿Desea buscar otro? ").lower()
            if opcion == 'si' or opcion == 'no':
                break
            else:
                print("Por favor, ingrese 'si' o 'no'.")
        if opcion != 'si':
            separador()
            break


    return

#### op3


In [8]:
def option_3():
    print("BUSCAR")
    separador()

    while True:
        if arbol.raiz is None:
            print("El árbol está vacío. No se puede buscar ningún nodo.")
            separador()
            break

        nombre_archivo = input("Ingrese el nombre del nodo a buscar: ")
        if not nombre_archivo:
            raise ValueError("El nombre del archivo no puede ser vacío")
        if not isinstance(nombre_archivo, str):
           raise TypeError("El nombre del archivo debe ser una cadena de caracteres")
        separador()

        nodo_encontrado = arbol.buscar(nombre_archivo)
        if nodo_encontrado:
            nivel = arbol.nivel(nodo_encontrado.nombre)
            if nivel is not None:
                print(f"Nivel del nodo: {nivel}")
            else:
                print(f"No se pudo obtener el nivel del nodo.")

            factor_balance = arbol.factor(nodo_encontrado)
            if factor_balance is not None:
                print(f"Factor de balance del nodo: {factor_balance}")
            else:
                print(f"No se pudo obtener el factor de balance del nodo.")

            padre = arbol.obtener_padre(nodo_encontrado)
            if padre is not None:
                print(f"Padre del nodo: {padre}")
            else:
                print(f"No se pudo obtener el padre del nodo.")

            abuelo = arbol.obtener_abuelo(nodo_encontrado)
            if abuelo is not None:
                print(f"Abuelo del nodo: {abuelo}")
            else:
                print(f"No se pudo obtener el abuelo del nodo.")

            tio = arbol.obtener_tio(nodo_encontrado)
            if tio is not None:
                print(f"Tío del nodo: {tio}")
            else:
                print(f"No se pudo obtener el tío del nodo.")
        else:
            print(f"No se encontró el nodo con nombre '{nombre_archivo}' en el árbol AVL.")

        while True:
            opcion = input("¿Desea buscar otro? ").lower()
            if opcion == 'si' or opcion == 'no':
                break
            else:
                print("Por favor, ingrese 'si' o 'no'.")
        if opcion != 'si':
            separador()
            break


#### op4

In [13]:
def obtener_nombre_imagenes_categoria(categoria, imagenes):
    nombres_imagenes = []
    for nombre_imagen in imagenes.get(categoria, []):
        nombres_imagenes.append(nombre_imagen["Nombre"])
    return nombres_imagenes


def buscar_tipo(nodo, tipo, rango_peso, nombres_images):
    if nodo is None:
        return []

    resultados = []

    if nodo.nombre is nombres_images and rango_peso[0]:
        resultados.append(nodo)

    resultados.extend(buscar_tipo(nodo.izq, tipo, rango_peso))
    resultados.extend(buscar_tipo(nodo.der, tipo, rango_peso))

    return resultados


In [10]:
def option_4():
    print("BUSCAR BY CRITERIA")
    separador()

    while True:
        tipo = input("Ingrese la categoría de las imágenes: ").lower()

        if not tipo:
            print("El tipo de archivo no puede estar vacío.")
        else:
            break

    while True:
        try:
            categoria = obtener_categoria(tipo)
            imagenes = procesar_imagenes(destino, carpetas_exist)
            nombres_imagenes_categoria = obtener_nombre_imagenes_categoria(categoria, imagenes)

            print(f"Imágenes de la categoría '{categoria}':")
            for nombre_imagen in nombres_imagenes_categoria:
                print(nombre_imagen)

            rango_min = int(input("Ingrese el valor mínimo del rango de peso: "))
            rango_max = int(input("Ingrese el valor máximo del rango de peso: "))

            if rango_min < 0 or rango_max < 0:
                print("Los límites del rango de peso deben ser números enteros positivos.")
            else:
                break
        except ValueError:
            print("Los límites del rango de peso deben ser números enteros.")

    rango_peso = (rango_min, rango_max)
    nodos_encontrados = buscar_tipo(arbol.raiz, tipo, rango_peso, nombres_imagenes_categoria)
    if nodos_encontrados:
        for nodo in nodos_encontrados:
            print(nodo.nombre)
        separador()
    else:
        print("No se encontraron archivos que cumplan con los criterios de búsqueda.")
        separador()



#### op5


In [11]:
def option_5():
    print("RECORRIDO POR NIVELES")
    separador()

    arbol.recorrido()
    separador()

#### main

In [14]:
# MAIN
print("Bienvenido al algoritmo del Árbol Binario Balanceado.")
separador()
print("Creadores: Juan Aguirre, Daniela Guerra, Maria Gabriela Reyes.")
separador()

#Validar nombre de user
#while True:
 #   user = input("Por favor, ingrese su nombre (debe tener más de 3 letras y no puede ser un número):\n")
  #  if len(user) > 3 and not user.isdigit():
   #     break
    #print("El nombre debe tener más de 3 letras y no puede ser un número. Inténtelo de nuevo.")

while True:
    print("Hola, ¿Que opcion te gustaria utilizar hoy?")
    print("1. Insertar nodo. ")
    print("2. Eliminar Nodo. ")
    print("3. Buscar Nodo.")
    print("4. Buscar Nodo (con parametros especificos).")
    print("5. Mostrar el recorrido por niveles del arbol.")
    print("6. Salir del programa.")

    while True:
        try:
            option = int(input("Seleccione una opción: "))
            if option < 1 or option > 6:
                raise ValueError("La opción debe ser un número del 1 al 6.")
            break
        except ValueError:
            print("Por favor, ingrese un número válido del 1 al 6.")
    separador()

    if option == 1:
        option_1(arbol, destino)

    elif option == 2:
        option_2()

    elif option == 3:
        option_3()

    elif option == 4:
        option_4()

    elif option == 5:
        option_5()

    elif option == 6:
        print("Eleccion escogida: [#6. Salir del programa y vaciar el árbol.]")
        arbol.vaciar_arbol()
        print("El árbol ha sido vaciado.")
        print("Hasta luego!")
        arbol.generar_dot_arbol('arbol.dot')

        separador()



Bienvenido al algoritmo del Árbol Binario Balanceado.
-----------------------------
Creadores: Juan Aguirre, Daniela Guerra, Maria Gabriela Reyes.
-----------------------------
Hola, ¿Que opcion te gustaria utilizar hoy?
1. Insertar nodo. 
2. Eliminar Nodo. 
3. Buscar Nodo.
4. Buscar Nodo (con parametros especificos).
5. Mostrar el recorrido por niveles del arbol.
6. Salir del programa.
Seleccione una opción: 4
-----------------------------
BUSCAR BY CRITERIA
-----------------------------
Ingrese la categoría de las imágenes: dog
Imágenes de la categoría 'dogs':
dog.2.jpg
dog.8.jpg
dog.7.jpg
dog.3.jpg
dog.6.jpg
dog.1.jpg
dog.4.jpg
dog.5.jpg
Ingrese el valor mínimo del rango de peso: 234
Ingrese el valor máximo del rango de peso: 32


TypeError: buscar_tipo() missing 1 required positional argument: 'nombres_images'